In [1]:
from datetime import datetime, timedelta
import pandas as pd
import json
import numpy as np
import logging
from azure.storage.blob import BlobServiceClient

# Set up the connection string and container names
connect_str = "*****Enter blob storage connection string*****"
input_container_name = '**** Enter container name where json files are stored *****'
output_container_name = '**** Enter container name to output CSV files *****'

blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_client = blob_service_client.get_container_client(input_container_name)
output_container_client = blob_service_client.get_container_client(output_container_name)


StatementMeta(mysparkpool, 6, 2, Finished, Available)

In [2]:
SYMBOLS = [
    'AAPL', 'ABBV', 'ABT', 'ACN', 'AIG', 'AMGN', 'AMT', 'AMZN', 'AXP', 'BA', 'BAC', 'BK', 'BKNG', 'BLK', 
    'BMY', 'BRK.B', 'C', 'CAT', 'CHTR', 'CL', 'CMCSA', 'COF', 'COP', 'COST', 'CRM', 'CSCO', 'CVS', 'CVX', 
    'DHR', 'DIS', 'DUK', 'DOW', 'EMR', 'EXC', 'F', 'FDX', 'GD', 'GE', 'GILD', 'GM', 'GOOGL', 'GS', 'HD', 
    'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KHC', 'KMI', 'KO', 'LMT', 'LOW', 'MA', 'MCD', 'MDLZ', 'MDT', 
    'MET', 'META', 'MMM', 'MO', 'MRK', 'MS', 'MSFT', 'NEE', 'NFLX', 'NKE', 'NVDA', 'ORCL', 'PEP', 'PFE', 
    'PG', 'PM', 'PYPL', 'QCOM', 'RTX', 'SBUX', 'SO', 'SPG', 'T', 'TGT', 'TJX', 'TMO', 'TMUS', 'TSLA', 
    'TXN', 'UNH', 'UNP', 'UPS', 'USB', 'V', 'VZ', 'WBA', 'WFC', 'WMT', 'XOM']

StatementMeta(mysparkpool, 6, 3, Finished, Available)

In [3]:
data = {
    'Symbol': [],
    'Name': [],
    'Industry': [],
    'EPS (fwd)': [],
    'P/E (fwd)': [],
    'PEG': [],
    'FCFY': [],
    'PB': [],
    'ROE': [],
    'P/S (trail)': [],
    'DPR': [],
    'DY': [],
    'CR': [],
    'Beta': [],
    'Price': [],
    '52w Low': [],
    '52w High': []
}

# Function to append data to dictionary
def load_data(json_data):
    data['Symbol'].append(json_data.get('symbol'))
    data['Name'].append(json_data.get('longName'))
    data['Industry'].append(json_data.get('industry'))
    data['Price'].append(json_data.get('currentPrice'))

    data['EPS (fwd)'].append(json_data.get('forwardEps', np.nan))
    data['P/E (fwd)'].append(json_data.get('forwardPE', np.nan))
    data['PEG'].append(json_data.get('pegRatio', np.nan))
    data['FCFY'].append(round((json_data.get('freeCashflow', 0) / json_data.get('marketCap', 1)) * 100, 2) if json_data.get('freeCashflow') and json_data.get('marketCap') else np.nan)
    data['PB'].append(json_data.get('priceToBook', np.nan))
    data['ROE'].append(json_data.get('returnOnEquity', np.nan))
    data['P/S (trail)'].append(json_data.get('priceToSalesTrailing12Months', np.nan))
    data['DPR'].append(json_data.get('payoutRatio', np.nan) * 100 if json_data.get('payoutRatio') else np.nan)
    data['DY'].append(json_data.get('dividendYield', 0.0))
    data['Beta'].append(json_data.get('beta', np.nan))
    data['CR'].append(json_data.get('currentRatio', np.nan))
    data['52w Low'].append(json_data.get('fiftyTwoWeekLow', np.nan))
    data['52w High'].append(json_data.get('fiftyTwoWeekHigh', np.nan))

# Read and parse each JSON file from the blob storage
for symbol in SYMBOLS:
    blob_name = f'{symbol}.json'
    blob_client = container_client.get_blob_client(blob_name)
    
    try:
        # Download the blob content
        blob_content = blob_client.download_blob().readall()
        json_data = json.loads(blob_content)
        load_data(json_data)
    except Exception as e:
        print(f"An error occurred while processing {symbol}: {e}")

# Create a DataFrame
fundamentals_df = pd.DataFrame(data)
fundamentals_df['52w Range'] = ((fundamentals_df['Price'] - fundamentals_df['52w Low']) / (fundamentals_df['52w High'] - fundamentals_df['52w Low'])) * 100

csv_data = fundamentals_df.to_csv(index=False)

# Upload the CSV data to the output container
output_blob_name = 'fundamentals.csv'
output_blob_client = output_container_client.get_blob_client(output_blob_name)
output_blob_client.upload_blob(csv_data, overwrite=True)

StatementMeta(mysparkpool, 6, 4, Finished, Available)

{'etag': '"0x8DC9930A006BE48"',
 'last_modified': datetime.datetime(2024, 6, 30, 18, 15, 30, tzinfo=datetime.timezone.utc),
 'content_md5': bytearray(b'\xd0\x18mzx\xba2\x9c\x05\xa8\x07\x13\xe4B\xd7V'),
 'client_request_id': 'bbd07ff6-370c-11ef-b9db-6045bd847db6',
 'request_id': '617caf08-401e-000e-1919-cbd993000000',
 'version': '2023-11-03',
 'version_id': None,
 'date': datetime.datetime(2024, 6, 30, 18, 15, 29, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None}